In [2]:
import torch
from torch_geometric.data import Data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
from torch.nn import Linear, Sequential, BatchNorm1d #, ReLU
from torch.nn.functional import relu
from torch.utils.data.sampler import SubsetRandomSampler
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_dense_adj
from torch_geometric.nn import ARMAConv
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import global_add_pool

import argparse
import time
import math
import copy

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdmolops
from rdkit.Chem import Crippen
from rdkit.Chem import PandasTools
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.rdchem import GetPeriodicTable
from rdkit import DataStructs
from rdkit.Chem import Descriptors
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.ML.Descriptors import MoleculeDescriptors
import rdkit

from mordred import Calculator, descriptors
from padelpy import padeldescriptor, from_smiles

import xgboost as xgb

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
#from sklearn.ensemble import RandomForestClassifier as RF
from sklearn import metrics
from sklearn.svm import SVR



In [3]:
xG_baruta_total_mordred_summation = pd.read_csv('../step2-applying_borutapy/old_xG_baruta_total_mordred_summation.csv')

In [4]:
total_smile = pd.read_csv('../dataset/total_smile.csv')
total_smile

,smile_A,mol_fraction_A,smile_B,mol_fraction_B
0,CCCCCCCCCCCCOS(=O)(=O)[O-],0.0,CCCCCCCCCCCCCCCC[N+](C)(C)C,1.0
1,CCCCCCCCCCCCOS(=O)(=O)[O-],0.1,CCCCCCCCCCCCCCCC[N+](C)(C)C,0.9
2,CCCCCCCCCCCCOS(=O)(=O)[O-],0.2,CCCCCCCCCCCCCCCC[N+](C)(C)C,0.8
3,CCCCCCCCCCCCOS(=O)(=O)[O-],0.3,CCCCCCCCCCCCCCCC[N+](C)(C)C,0.7
4,CCCCCCCCCCCCOS(=O)(=O)[O-],0.4,CCCCCCCCCCCCCCCC[N+](C)(C)C,0.6
...,...,...,...,...
974,CCCCCCCCCCCCCCOCC(O)Cn1cc[n+](CCC[n+]2ccn(CC(O...,0.5,CCCCCCCCCCCCCCOCC(O)Cn1cc[n+](CCC[n+]2ccn(CC(O...,0.5
975,CCCCCCCCCCCCCCC(CSCCSCC(CCCCCCCCCCCCCC)[n+]1cc...,0.5,CCCCCCCCCCCCCCC(CSCCSCC(CCCCCCCCCCCCCC)[n+]1cc...,0.5
976,CCCCCCCCCCCCCCC(CSCCCSCC(CCCCCCCCCCCCCC)[n+]1c...,0.5,CCCCCCCCCCCCCCC(CSCCCSCC(CCCCCCCCCCCCCC)[n+]1c...,0.5
977,CCCCCCCCCCCCCCC(CSCCCCSCC(CCCCCCCCCCCCCC)[n+]1...,0.5,CCCCCCCCCCCCCCC(CSCCCCSCC(CCCCCCCCCCCCCC)[n+]1...,0.5


In [5]:
Z_total = pd.read_csv('../step1-generating and merging descriptors/total_property.csv')

In [6]:
unique_chem = pd.read_csv('../dataset/mixture_diff_chem.csv')
unique_chem

,smile_A,mol_fraction_A,smile_B,mol_fraction_B
0,CCCCCCCCCCCCOS(=O)(=O)[O-],0.40,CCCCCCCCCCCCCCCC[N+](C)(C)C,0.60
1,CCCCCCCCCCCCOS(=O)(=O)[O-],0.40,CCCCCCCCCCOS(=O)(=O)[O-],0.60
2,CCCCCCCCCCCCOS(=O)(=O)[O-],0.40,CCCCCCCCCCS(C)=O,0.60
3,CCCCCCCCCCOS(=O)(=O)[O-],0.48,CCCCCCCCCC[N+](C)(C)C,0.52
4,CCCCCCCCCCCCOS(=O)(=O)[O-],0.40,CCCCCCCC(=O)N(C)CC(O)C(O)C(O)C(O)CO,0.60
...,...,...,...,...
61,CCCCCCCCCCS(=O)(=O)[O-],0.50,CC(C)CC(C)(C#CC(C)(CC(C)C)OCCOCCOCCOCCOCCO)OCC...,0.50
62,CCCCCCCCCCCCS(=O)(=O)[O-],0.50,CC(C)CC(C)(C#CC(C)(CC(C)C)OCCOCCOCCOCCOCCO)OCC...,0.50
63,CCCCC(CC)CCC(CC(C)C)S(=O)(=O)[O-],0.50,CC(C)CC(C)(C#CC(C)(CC(C)C)OCCOCCOCCOCCOCCO)OCC...,0.50
64,O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(...,0.50,CCCCCCCCn1cc[n+](C)c1,0.50


In [7]:
smile_A = unique_chem['smile_A']
smile_B = unique_chem['smile_B']

mixture_train_smile_A, mixture_test_smile_A, mixture_train_smile_B, mixture_test_smile_B = train_test_split(smile_A, smile_B, test_size=0.2,shuffle=True,random_state=42)

In [8]:
mixture_train_smile_A.shape, mixture_test_smile_A.shape

((52,), (14,))

In [9]:
for A, B in zip(mixture_train_smile_A, mixture_test_smile_A):
    print(A, B)

CCCCCCCCCCCCOCCOCCOCCOCCOCCOP(=O)([O-])O CCCCCCCCCCCCOS(=O)(=O)[O-]
O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F CCCCCCCCCCCCS(=O)(=O)[O-]
CCCCCCCCCCOS(=O)(=O)[O-] CCCCCCCCCCCCOS(=O)(=O)[O-]
CC(CCC(=O)[O-])C1CCC2C3CCC4CC(O)CCC4(C)C3CC(O)C12C CCCCCCCCCCCCCC[N+](C)(C)C
CCCCCCCCCCCC[N+](C)(C)CC(=O)[O-] CCCCCCCCCCCCOS(=O)(=O)[O-]
CCCCCCCCCCP(C)(C)=O CCCCC(CC)CCC(CC(C)C)S(=O)(=O)[O-]
CCCCCCCCCCCCOS(=O)(=O)[O-] CC(CCC(=O)[O-])C1CCC2C3CCC4CC(O)CCC4(C)C3CC(O)C12C
O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F CCCCCC(=O)[O-]
CCCCCCCCCCCCOS(=O)(=O)[O-] O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F
CCCCCCCCCCCCCC[N+](C)(C)C CCCCC(CC)COC(=O)CC(C(=O)OCC(CC)CCCC)S(=O)(=O)[O-]
CCCCCCCCCCCCOCCOCCOCCOCCOCCOP(=O)([O-])O CCCCCCCCC(C)CCCOCCOCCOCCOCCOCCOP(=O)([O-])O
C[N+](CCO)(CCCO)CC(O)CC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F CCCCCCCCCCCC[N+](C)(C)C
CCCCCCCCCCC(Oc1ccc(OC(CCCCCCCCCC)C(=O)[O-])cc1)C(=O)[O-] CCCCCCCCCCCCOS(=O)(=O)[O-]
CCCCCCCCCCCCOS(=

In [10]:
# Filter the total_smile dataframe to only include rows where 'smile_A' matches 'test_smile_A' and .....
# 'smile_B' matches 'test_smile_B'

test_smile = total_smile[
    (total_smile['smile_A'].isin(mixture_test_smile_A)) & 
    (total_smile['smile_B'].isin(mixture_test_smile_B))
]

# Save the corresponding indices of test rows
test_matching_indices = total_smile[
    (total_smile['smile_A'].isin(mixture_test_smile_A)) & 
    (total_smile['smile_B'].isin(mixture_test_smile_B))
].index

# Drop the test rows from total_smile dataframe to get train_smiles
train_smile = total_smile.drop(test_matching_indices)

test_smile.to_csv('test_smile.csv', index=False)
train_smile.to_csv('train_smile.csv', index=False)

train_smile.shape, test_smile.shape

((844, 4), (135, 4))

In [12]:
xG_baruta_test_mordred_summation = xG_baruta_total_mordred_summation.loc[test_matching_indices]
xG_baruta_train_mordred_summation = xG_baruta_total_mordred_summation.drop(test_matching_indices)

xG_baruta_test_mordred_summation.to_csv('xG_baruta_test_mordred_summation.csv', index=False)
xG_baruta_train_mordred_summation.to_csv('xG_baruta_train_mordred_summation.csv', index=False)

In [13]:
Z_total = pd.read_csv('../total_property.csv')

# Use these indices to filter the 'Z_total' dataframe
Z_test = Z_total.loc[test_matching_indices]

# Save the rest of the rows in 'Z_train'
Z_train = Z_total.drop(test_matching_indices)

Z_test.to_csv('test_property.csv', index=False)
Z_train.to_csv('train_property.csv', index=False)

Z_train.shape,Z_test.shape

((844, 1), (135, 1))